# Exploration of chemical pollutation from a place based perspective

{bdg-link-info}`Notebook Repository <https://github.com/NERC-CEH/ds-toolbox-notebook-EEX-placebased-exposure.git>`
{bdg-warning-line}`Ongoing Development`
![alt text](./images/ukceh-logo-badge.png "Title")

Primary Contact: [Dr. Michael Hollaway](https://www.ceh.ac.uk/staff/michael-hollaway)

````{card} Challenge:
When thinking about a particular place (E.g. a particular point, town, city or even a region), people often want to know more about the levels of pollution in that region in the present and often the past. This presents a significant challenge as there are multiple sources of pollutions (e.g. air pollution or water pollution) and multiple providers/collectors of pollution data (e.g. the Environment Agency (EA) or the Department for Environment, Food and Rural Affairs (Defra)). In addition there are ever increasing volumes of data being collected from a variety of sensor types. If someone wants to answer the question of "I lived in the North East of England from 2000 to 2025, what sort of pollution was I exposed to?" a significant challenges is presented to extract, fileter, aggregate and visualised the data in a meaningful way. The resultant workflow often requires different tools and technical skills to be able to answer the question. 
````

````{card} Approach:
This notebook presents a demonstration of one potential approach to addressing this challenge using examples of data from the Defra managed Automatic Urban and Rural Network ([AURN] (https://uk-air.defra.gov.uk/networks/network-info?view=aurn)) and the EA managed Water quality Gas/Liquid Chromatography mass spectrometry ([GCMS/LCMS](https://www.data.gov.uk/dataset/0c63b33e-0e34-45bb-a779-16a8c3a4b3f7/water-quality-monitoring-data-gc-ms-and-lc-ms-semi-quantitative-screen)) datasets to represent air and water pollution respectively. Developed using the python coding language the code shows methods for choosing a particular place, extracting all the available data for that place, calculating the summary statistics for that region (including some simple data science methods to interpolate between measurement locations) and visualising the final results. Finally functiionality is provided to run the notebook as a dashboard allowing the user to interatively run the analysis over different places. 
````

```{admonition} Running the Notebook:
:class: tip, dropdown
To run the notebook it is advised to first clone the repository housing the notebook ('*git clone https://github.com/NERC-CEH/ds-toolbox-notebook-EEX-placebased-exposure.git*'). This will create a folder in the current directory called *ds-toolbox-notebook-EEX-placebased-exposure*, which holds all the relevant files including the notebook, environment file and relevant input data. The next step is to create a conda environment with the right packages installed using the clean yml file ('*conda env create -f environment_clean.yml*'), which creates the *EEX-placebased-exposure* environment that can be activated using '*conda activate EEX-placebased-exposure*'. At this point the user can either run code from the notebook in their preferred IDE or via the jupyter interface using the command '*jupyter notebook*'.
```

```{admonition} Generalisability:
:class: note, dropdown
The methods and processes demonstrated in this notebook should be transferable to other datasets beyond the air quality and water quality examples used here. These methods should be able to work with other point based datasets and the spatial filtering should work for other polygons. The workflow presented is designed to be a starter example for a potential method to investigate location specific chemical pollution and is setup to be easily built upon for more detailed analysis.   
```

```{admonition} Data Sources:
:class: note, dropdown
This notebook uses data from the following sources:

### Air Quality Data: Automatic Urban and Rural Network (AURN) 

A description of the AURN network is available from the \href{https://uk-air.defra.gov.uk/networks/network-info?view=aurn}{UK-AIR}
website.The data are freely available to download and are provided by the Department for the Environment and Rural Affairs
(Defra) under the following attribution statement:

(C) Crown copyright 2021 Defra via uk-air.defra.gov.uk, licensed under the \href{https://www.nationalarchives.gov.uk/doc/open-government-licence/version/3/}{Open Government Licence}.

### Water Quality Data: Water quality monitoring data GC-MS and LC-MS semi-quantitative screen

A description of the EA water quality GC-MS and LC-MS semi-quantitative screen data is available from the \href{https://www.gov.uk/government/publications/water-quality-monitoring-data-gc-ms-and-lc-ms-semi-quantitative-screen}{data.gov.uk}

This dataset is licensed under the \href{https://www.nationalarchives.gov.uk/doc/open-government-licence/version/3/}{Open Government Licence}.


```

```{admonition} Computational Demands
:class: warning, dropdown
It should be noted that whilst this notebook in its current form runs fairly quickly on a standard laptop computer, this is achieved by pre-processing the raw data files into parquet format and only demonstrating the functionality of the methods using fairly computationally light operations and on a relatively small dataset. Therefore the user should be aware that if they desire to bring in more complex or advanced analytical methods or larger datasets then the computational demand will likely increase and appropriate CPU/GPU time may be needed. 
```

```{contents}
:local:
```
<!-- https://jupyterbook.org/en/stable/structure/configure.html -->

In [ ]:
import polars as pl
import pandas as pd
import holoviews as hv
import geoviews as gv
import geopandas as gpd
from cartopy import crs as ccrs
import numpy as np
from holoviews.streams import Selection1D
import panel as pn
from scipy.interpolate import Rbf, griddata
import matplotlib.pyplot as plt

In [ ]:
#Set up the bokeh extensions.
gv.extension('bokeh')
hv.extension('bokeh')
pn.extension()

In [ ]:
#Set the location of the data.
data_path = "C:\\Users\\mhollaway\\Documents\\JNCC_SLI2_tool\\Test_data\\"

#Read in the NUTS regions - NUTS3 in this example.
NUTS_gdf = gpd.read_file(data_path + 'NUTS_Level_1_January_2018_GCB_in_the_United_Kingdom_2022_-2753267915301604886.geojson')

#Read in the pre-processed data.
all_data       = pl.read_parquet(data_path + "EEX_all_data.parquet")
Sites_info_gdf = gpd.read_parquet(data_path + "Sites_info_gdf.parquet")

In [ ]:
NUTS_plt = gv.Polygons(NUTS_gdf, vdims=['nuts118cd','nuts118nm']).opts(color='black', fill_alpha=0, projection=ccrs.PlateCarree(),width=600,height=1000, tools=['hover', 'tap'], active_tools=['tap'], shared_axes=False)


In [ ]:
#Function to calculate a summary of the chemical data in a selected region. 
def calc_chem_summary(index):

    #Check for empty index if no selection yet made.
    if not index:
        out_col_list = ['Pollutant', 'Min Concentration', 'Max Concentration', 'Mean Concentration',
                        'Number of Sites', 'Measurement Category', 'Unit', 'Measurement Source',
                        'Sample Type', 'Date of first measurement', 'Date of last measurement']
        return hv.Table(pd.DataFrame(columns=out_col_list)).opts(width=1200, height=400)

    #If we have an index pick the geometry for filtering.
    selected_poly = NUTS_gdf.geometry.iloc[index[0]]

    # Filter sites in a specific polygon - this will be picked up using the index from the selector.
    Sites_in_poly = Sites_info_gdf['Site_ID'].loc[Sites_info_gdf.geometry.within(selected_poly)]

    #If no chemicals for given site pass back empty table.
    if Sites_in_poly.shape[0] == 0:
        out_col_list = ['Pollutant', 'Min Concentration', 'Max Concentration', 'Mean Concentration',
                        'Number of Sites', 'Measurement Category', 'Unit', 'Measurement Source',
                        'Sample Type', 'Date of first measurement', 'Date of last measurement']
        return hv.Table(pd.DataFrame(columns=out_col_list)).opts(width=1200, height=400)
    else:

        # Get the data.
        curr_rgn_data = all_data.filter((pl.col('Site_ID').is_in(Sites_in_poly) == True) & (pl.col('Concentration') > 0.0))

        #Get the required summary for the region.
        summ_rgn_data = curr_rgn_data.group_by('Pollutant').agg([
            pl.col('Concentration').min().alias('Min Concentration'),
            pl.col('Concentration').max().alias('Max Concentration'),
            pl.col('Concentration').mean().alias('Mean Concentration'),
            pl.col('Site_ID').n_unique().alias('Number of Sites'),
            pl.col('Measurement Category').unique().alias('Measurement Category'),
            pl.col('Unit').unique().alias('Unit'),
            pl.col('Source').unique().alias('Measurement Source'),
            pl.col('Site_Type').unique().alias('Sample Type'),
            pl.col('Date').min().alias('Date of first measurement'),
            pl.col('Date').max().alias('Date of last measurement')])

        #Format the correct output for columns where multiple categories are returned.
        summ_rgn_data = summ_rgn_data.with_columns(pl.col('Unit').list.join(',').alias('Unit'),
                                                   pl.col('Measurement Source').list.join(',').alias('Measurement Source'),
                                                   pl.col('Sample Type').list.join(',').alias('Sample Type'))


        #Return either as a holoviews table (if in a panel app) or a pandas dataframe (if in the notebook)
        if pn.state.served == True:
            summ_reg_table = hv.Table(summ_rgn_data.to_pandas()).opts(width=1200, height=400)
        else:
            summ_reg_table = summ_rgn_data.to_pandas()
        
        return summ_reg_table

In [ ]:
calc_chem_summary([0])

In [ ]:
def plot_chem_sites(index, chem):

    #If no selection made display nothing.
    if not index:
        return gv.Polygons([], label='Please select a polygon to view data').opts(width=800,height=400, shared_axes=False)

    # If we have an index pick the geometry for filtering.
    selected_poly = NUTS_gdf.geometry.iloc[index[0]]
    Sites_in_poly = Sites_info_gdf[['Site_ID', 'geometry']].loc[Sites_info_gdf.geometry.within(selected_poly)]

    # Get the data.
    curr_rgn_data = all_data.filter(pl.col('Site_ID').is_in(Sites_in_poly['Site_ID']) == True).select(['Site_ID', 'Pollutant', 'Concentration','Measurement Category'])

    #If no data for the selected chemical return nothing.
    if curr_rgn_data.shape[0] == 0:
        if pn.state.served == True:
            return gv.Polygons([], label='No Data for selected chemical for this region.').opts(width=800,height=400, shared_axes=False)

    #Run a check on data type to plot, for vet meds plot locations and heatmap by samples, if AQ plot spatial distribution based on measurements.
    if curr_rgn_data.filter(pl.col('Pollutant') == chem).select('Measurement Category').unique().item() == 'Vet meds':

        count_df = curr_rgn_data.filter(pl.col('Pollutant') == chem).group_by('Site_ID').agg([pl.col('Concentration').mean().alias('Mean_Concentration')])

        curr_chem_pts = Sites_in_poly.merge(count_df.to_pandas(), on='Site_ID')

        #Now create the map.
        #If in panel app pass out interactive plot if in regular notebook pass out static plot.
        if pn.state.served == True:

            #Polygons first.
            curr_poly_layer = gv.Polygons(selected_poly).opts(color='black', fill_alpha=0, line_width=2, projection=ccrs.PlateCarree()) #Fill alpha sets fill to transparent.!
            #Create a heatmap of points based on number of samples.
            curr_pts_layer  = gv.Points(curr_chem_pts, vdims=['Site_ID','Mean_Concentration']).opts(
                                        size=10,
                                        color='Mean_Concentration',
                                        cmap='Viridis',
                                        tools=['hover'],
                                        width=600,
                                        height=400,
                                        title='Mean Concentration of ' + chem + '(2000-2025) for: ' + NUTS_gdf['nuts118nm'].iloc[index[0]]
                                        )

            #Create the combined plot and set the dimensions.
            curr_chem_plot  = (curr_poly_layer * curr_pts_layer).opts(width=800,height=400, shared_axes=False)

        else:

            #Plot the data as a staic matplotlib plot if not in a panel app.        
            fig, ax = plt.subplots(figsize=(7, 5))

            #Polygon.
            NUTS_gdf.iloc[[index[0]]].plot(ax=ax, facecolor='none', edgecolor='black')

            # Overlay points
            points = ax.scatter(curr_chem_pts.geometry.x, curr_chem_pts.geometry.y, c=curr_chem_pts.Mean_Concentration, cmap='viridis', s=50, edgecolor='black', label='Measurement Sites')
            fig.colorbar(points, ax=ax, label='Mean Concentration')

            # Add labels and legend
            ax.set_xlabel('Longitude')
            ax.set_ylabel('Latitude')

            #Set the title.
            ax.set_title('Mean Concentration of ' + chem + '(2000-2025) for: ' + NUTS_gdf['nuts118nm'].iloc[index[0]])

            curr_chem_plot = fig



    elif curr_rgn_data.filter(pl.col('Pollutant') == chem).select('Measurement Category').unique().item() == 'Air quality':

        #Calculate the total exposure to the air pollutant.
        total_exp_df  = curr_rgn_data.filter(pl.col('Pollutant') == chem).group_by('Site_ID').agg([pl.col('Concentration').mean().alias('Mean Exposure')])
        curr_chem_pts = Sites_in_poly.merge(total_exp_df.to_pandas(), on='Site_ID')
        #Filter on where we have data.
        curr_chem_pts = curr_chem_pts.loc[(curr_chem_pts['Mean Exposure'] > 0.0) & (curr_chem_pts['Mean Exposure'].isnull() == False)]

        # KDE heatmap
        x = curr_chem_pts.geometry.x
        y = curr_chem_pts.geometry.y
        weights = curr_chem_pts['Mean Exposure'].values

        # Grid over polygon bounds - this will be used to model the exposure across the region on.
        xmin, ymin, xmax, ymax = selected_poly.bounds
        xx, yy    = np.mgrid[xmin:xmax:100j, ymin:ymax:100j] #Meshgrid for modelling the KDE on

        # Quick check to see if we have enough data to fit the interpolation - if not just pass out zeros.
        if len(weights) >= 4:
          density = griddata(np.column_stack([x,y]), weights, (xx, yy), method='linear')
        else:
          density = np.zeros(xx.shape)

        #Depending on if running in a notebook render as either holoviews (panel app) or matplotlib (notebook).
        if pn.state.served == True:

            # Holoviews Image
            AQ_heatmap = hv.QuadMesh((xx,yy,density)).opts(cmap='Viridis',alpha=0.6,tools=['hover'],line_color=None,line_width=1,
                                                        width=600,
                                                        height=400,title='Air pollutant exposure: ' + chem)

            #Also add the locations of the sampling points for the current AQ pollutant.
            curr_pts_layer = gv.Points(curr_chem_pts, vdims=['Site_ID']).opts(
                size=10,
                color='Black',
                tools=['hover'],
                width=600,
                height=400,
                title='Sample Density Heatmap'
            )


            #Polygon layer.
            curr_poly_layer = gv.Polygons(selected_poly).opts(color='black', fill_alpha=0, line_width=2,projection=ccrs.PlateCarree())  # Fill alpha sets fill to transparent.!

            #Produce the final plot.
            curr_chem_plot = (curr_poly_layer * AQ_heatmap * curr_pts_layer).opts(width=600, height=400, shared_axes=False)

        else:

            fig, ax = plt.subplots(figsize=(7, 5))

            #Plot the data as a staic matplotlib plot if not in a panel app.
            mesh = ax.pcolormesh(xx, yy, density, cmap='viridis', shading='auto')
            fig.colorbar(mesh, ax=ax, label='Total Concentration')

            #Polygon.
            NUTS_gdf.iloc[[index[0]]].plot(ax=ax, facecolor='none', edgecolor='black')

            # Overlay points
            ax.scatter(curr_chem_pts.geometry.x, curr_chem_pts.geometry.y, color='black', s=50, edgecolor='black', label='Measurement Sites')

            # Add labels and legend
            ax.set_xlabel('Longitude')
            ax.set_ylabel('Latitude')

            #Set the title.
            ax.set_title('Air pollutant exposure: ' + chem + ' for: ' + NUTS_gdf['nuts118nm'].iloc[index[0]])

            curr_chem_plot = fig

    return curr_chem_plot

In [ ]:
plot_chem_sites([0], 'no2')

In [ ]:
#Only run this cell if in a panel app.
if pn.state.served == True:

    #Now set up the notebook to run interactively if served via panel.
    #Create a selection stream - this will be used to pick up the chemical data for each region.
    selection_summary = Selection1D(source=NUTS_plt)

    #Create a panel widget that holds the list of available chemicals.
    chem_options = pn.widgets.Select(name='List of Chemicals', options=[])

    #Callback function to update the widget.
    #Need to se watch to true here as panel is updating the widget.
    @pn.depends(selection_summary.param.index, watch=True)
    def update_chem_options(index):

        if index:
            selected_poly        = NUTS_gdf.geometry.iloc[index[0]]
            Sites_in_poly        = Sites_info_gdf['Site_ID'].loc[Sites_info_gdf.geometry.within(selected_poly)]
            if Sites_in_poly.shape[0] == 0:
                chem_options.options = ['No Chemicals Found']
                chem_options.value = 'No Chemicals Found'
            else:
                chem_options.options = all_data.filter(pl.col('Site_ID').is_in(Sites_in_poly) == True).get_column('Pollutant').unique().to_list()
                chem_options.value   = all_data.filter(pl.col('Site_ID').is_in(Sites_in_poly) == True).get_column('Pollutant').unique().to_list()[0]
        else:
            chem_options.options = ['No Chemicals Found']
            chem_options.value = 'No Chemicals Found'

    #Now need to set up reactive versions of the above functions to work in panel.
    chem_summary_reactive = pn.bind(calc_chem_summary, selection_summary.param.index)
    chem_plot_reactive    = pn.bind(plot_chem_sites, selection_summary.param.index, chem_options.param.value)

    #Create the panel layout and serve.
    dashboard = pn.Row(pn.Column(NUTS_plt) , pn.Column(pn.panel(chem_summary_reactive), pn.panel(chem_options), pn.panel(chem_plot_reactive)))
    dashboard.servable()
